# **genomeNLP: Case study of deep learning**

[![Binder](https://binderhub.rc.nectar.org.au/badge_logo.svg)](https://binderhub.rc.nectar.org.au/v2/gh/tyronechen/genomenlp.git/HEAD?labpath=src%2Fjupyter%2Fcase_study_dna.ipynb)
[![](https://readthedocs.org/projects/genomenlp/badge/?version=latest&style=for-the-badge)](https://genomenlp.readthedocs.io/en/latest/)
[![](https://flat.badgen.net/badge/icon/gitlab?icon=gitlab&label&color=orange&scale=1.5)](https://gitlab.com/tyagilab/genomenlp)
[![](https://flat.badgen.net/badge/icon/github?icon=github&label&color=black&scale=1.5)](https://github.com/tyronechen/genomenlp)

Copyright (c) 2023 [Tyrone Chen](https://orcid.org/0000-0002-9207-0385)![image.png](https://info.orcid.org/wp-content/uploads/2019/11/orcid_16x16.png"), [Navya Tyagi](https://orcid.org/0000-0002-8797-3168)![image.png](https://info.orcid.org/wp-content/uploads/2019/11/orcid_16x16.png") and [Sonika Tyagi](https://orcid.org/0000-0003-0181-6258)![image.png](https://info.orcid.org/wp-content/uploads/2019/11/orcid_16x16.png").

Code in this repository is provided under a [MIT license](https://opensource.org/license/mit/). Documentation for this specific case study is provided with © all rights reserved (temporary until publication). All other documentation not on this page is provided under a [CC-BY-3.0 AU license](https://creativecommons.org/licenses/by/3.0/au/).

## **Outline**
The primary focus of this tutorial is application of NLP in a genomic context by introducing our package `genomenlp`. In this tutorial, we cover a wide range of topics from introduction to field of GenomeNLP to practical application skills of our `conda` package, divided into various sections:

1. Introduction to GenomeNLP
2. Connection to a remote server
3. Installing conda and genomenlp package
4. Setting up a Biological Dataset
5. Format a dataset as input for genomenlp
6. Preparing a hyperparameter sweep
7. Selecting optimal parameters
8. With the selected hyperparameters, train the full dataset
9. Performing cross-validation
10. Comparing performance of different models
11. Obtain model interpretability scores

For detailed usage of individual functions, please refer to the latest documentation.

### **Learning objectives**
- Describe the unique challenges in biological NLP compared to conventional NLP
- Understand common representations of biological data
- Understand common biological data preprocessing steps
- Investigate biological sequence data for use in machine learning
- Perform a hyperparameter sweep, training and cross-validation
- Identify what the model is focusing on
- Compare trained model performances to each other

` NOTE: This is **not** an introductory machine learning workshop.
  Readers of this tutorial are assumed to be familiar with
  the use of the command line and of the basics of machine learning.`

### **Potential/preferred prerequisite knowledge**

- [required] CLI (e.g. bash shell) usage
- [optional] Connecting and working on a remote server (e.g. ``ssh``)
- [optional] Basic knowledge of machine learning
- [optional] Machine learning dashboards (e.g. ``tensorboard``, ``wandb``)
- [optional] Package/environment managers (e.g. ``conda``, ``mamba``)

Length: Half-day, 4.0 - 4.5 hours

Intended audience: machine learning practitioners OR computational biologists

### **Glossary**
- BERT - Bidirectional Encoder Representations from Transformers, a family of deep learning architectures used for NLP.
- DL - Deep Learning
- k-mers - Identical to tokens
- k-merisation - A process where a biological sequence is segmented into substrings. Commonly performed as a sliding window.
- ML - Machine Learning
- NLP - Natural Language Processing
- OOV - Out-of-vocabulary words
- Sliding window - ABCDEF: [ABC, BCD, CDE, DEF] instead of [ABC, DEF]
- Tokenisation - A process where a string is segmented into substrings
- Tokens - Subunits of a string used as input into conventional NLP algorithms



## 1. **Introduction**

### **What is NLP and genomics**
Natural Language Processing (NLP) is a branch of computer science focused around the understanding of and the processing of human language. Such a task is non-trivial, due to the high variation in meaning of words found embedded in different contexts. Nevertheless, NLP is applied with varying degrees of success in various fields, including speech recognition, machine translation and information extraction. A recent well-known example is ChatGPT.

![image.png](https://genomenlp.readthedocs.io/en/latest/_images/applications_example.png)

Meanwhile, genomics involves the study of the genome, which contains the entire genetic content of an organism. As the primary blueprint, it is an important source of information and underpins all biological experiments, directly or indirectly.

### **Why apply NLP in genomics**
Although NLP has been shown to effectively preprocess and extract “meaning” from human language, until recently, its application in biology was mostly centred around biological literature and electronic health record mining. However, we note the striking similarities between genomic sequence data and human languages that make it well-suited to NLP. (A) DNA can be directly expressed as human language, being composed of text strings such as A, C, T, G, and having its own semantics as well as grammar. (B) Large quantities of biological data are available in the public domain, with a growth rate exponentially exceeding astronomy and social media platforms combined. (C) Recent advances in machine learning which improve the scalability of deep learning (DL) make computational analysis of genomic data feasible.

`NOTE: The same is true for protein sequences, and nucleic acid data such as
transcripts. While our pipeline can process any of these, the scope of
this tutorial is for genomic data only.`

![image.png](https://genomenlp.readthedocs.io/en/latest/_images/data_growth.png)

We therefore make a distinction between the field of conventional literature or electronic health record mining and the application of NLP concepts and methods to the genome. We call this field `genome NLP`. The aim of `genome NLP` would be to extract relevant information from the large corpora of biological data generated by experiments, such as gene names, point mutations, protein interactions and biological pathways. Applying concepts used in NLP can potentially enhance the analysis and interpretation of genomic data, with implications for research in personalised medicine, drug discovery and disease diagnosis.

### **Distinction between conventional NLP and genome NLP**

Several key differences need to be accounted for for implementing NLP on the genome. (A) The first challenge is the tokenisation of long biological sequences into smaller subunits. While some natural languages have subunits separated by spaces, enabling easy segmentation, this is not true in biological sequence data, and also to an extent in many languages such as Arabic, Mandarin or Sanskrit characters. (B) A second challenge is the diversity and high degree in nuance of biological experiments. As a result, interpretability and interoperability of biological data is highly restricted in scope, even within a single experiment. (C) The third challenge is the difficulty in comparing models, partly due to the second challenge, and partly due to the lack of accessible data in the biomedical field for privacy reasons, and partly because of the [limited enforcement of biological data integrity as well as metadata by journals](https://academic.oup.com/view-large/figure/129641572/gky1064fig3.jpg?login=false). In addition, the large volume of biological data in a single experiment makes re-training time consuming.

![image.png](https://genomenlp.readthedocs.io/en/latest/_images/bio_vs_nlp.png)

To address the challenges in genome-NLP, we used a new semi-automated workflow. This workflow integrates feature engineering and machine learning techniques and is designed to be adaptable across different species and biological sequences, including nucleic acids and proteins. The workflow includes the introduction of a (1) new tokeniser for biological sequence data which effectively tokenises contiguous genomic sequences while retaining biological context. This minimises manual preprocessing, reduces vocabulary sizes, and (2) handles unknown biological terms, conceptually identical to the out-of-vocabulary (OOV) problem in natural languages. (3) Passing the preprocessed data to a `genomicBERT` algorithm allows for direct biological sequence input to a state-of-the-art deep learning algorithm. (4) We also enable model comparison by weights, removing the need for computationally expensive re-training or access to raw data. To promote collaboration and adoption, `genomicBERT` is available as part of the publicly accessible `conda` package called `genomeNLP`. [Successful case studies](https://zenodo.org/record/8324849) have demonstrated the effectiveness of `genomeNLP` in genome NLP applications.

![image.png](https://genomenlp.readthedocs.io/en/latest/_images/graphical_abstract.png)








## 2. Connect to a remote server

To standardise the compute environment for all participants, we will be establishing a network connection with a remote server. Data and a working install of `genomenl`p is provided. Secure Shell (SSH) is a common method for remote server connection, providing secure access and remote command execution through encrypted connections between the client and server.

To use `ssh` (Secure Shell) for remote server access, please follow these steps:
1. Open a Terminal or Command Prompt on your local machine. SSH is typically available on Unix-like systems (e.g. Linux, macOS) and can also be installed on Windows systems using tools like [PuTTY](https://www.chiark.greenend.org.uk/~sgtatham/putty/latest.html) or [MobaXterm](https://mobaxterm.mobatek.net/download.html).
2. Determine the ssh command syntax. Generally the format is: `ssh username@hostname` or the IP address of the remote server.
3. Enter your password or passphrase when prompted. Once authenticated, you should be connected to the remote server via SSH.

`NOTE: Details for (2) and (3) will be provided on the day of the workshop.`




## 3. Installing conda, mamba and genomenlp

`NOTE: This step is already performed for you. Information is provided as a guide for those who are reading this document outside of the tutorial, or if for some reason the installation is not working.`

A package/environment manager is a software tool that automates the installation, update, and removal of packages and allows for the creation of isolated environments with specific configurations. This simplifies software setup, reduces compatibility issues, and improves software development workflows. Popular examples include `apt` and `anaconda`. We will use `conda` and `mamba` in this case study.

`NOTE: The same is true for protein sequences, and nucleic acid data such as transcripts. While our pipeline can process any of these, the scope of this tutorial is for genomic data only.`

To install `conda` using the command line, you can follow these steps:

1. Open your command prompt. Use the `curl` or `wget` command to download the installer directly from the command line using its URL.

In [ ]:
! wget -O miniconda.sh https://repo.anaconda.com/miniconda/Miniconda3-py37_4.10.3-Linux-x86_64.sh
! chmod +x miniconda.sh

--2023-10-04 03:01:18--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.10.3-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89026327 (85M) [application/x-sh]
Saving to: ‘miniconda.sh’

miniconda.sh        100%[===================>]  84.90M   229MB/s    in 0.4s    

2023-10-04 03:01:19 (229 MB/s) - ‘miniconda.sh’ saved [89026327/89026327]



2. Run the installer script using the following command:

In [ ]:
! bash ./miniconda.sh -b -f -p /usr/local
! rm miniconda.sh

PREFIX=/usr/local
Unpacking payload ...
Solving environment: | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - _openmp_mutex==4.5=1_gnu
    - brotlipy==0.7.0=py37h27cfd23_1003
    - ca-certificates==2021.7.5=h06a4308_1
    - certifi==2021.5.30=py37h06a4308_0
    - cffi==1.14.6=py37h400218f_0
    - chardet==4.0.0=py37h06a4308_1003
    - conda-package-handling==1.7.3=py37h27cfd23_1
    - conda==4.10.3=py37h06a4308_0
    - cryptography==3.4.7=py37hd23ed53_0
    - idna==2.10=pyhd3eb1b0_0
    - ld_impl_linux-64==2.35.1=h7274673_9
    - libffi==3.3=he6710b0_2
    - libgcc-ng==9.3.0=h5101ec6_17
    - libgomp==9.3.0=h5101ec6_17
    - libstdcxx-ng==9.3.0=hd4cf53a_17
    - ncurses==6.2=he6710b0_1
    - openssl==1.1.1k=h27cfd23_0
    - pip==21.1.3=py37h06a4308_0
    - pycosat==0.6.3=py37h27cfd23_0
    - pycparser==2.20=py_2
    - pyopenssl==20.0.1=pyhd3eb1b0_1
    - pysocks==1.7.1=py37_1
    - python==3.7.10=h12debd9

3. To install conda and mamba, run the following command:

If you are performing this in google colab, please allow 10 minutes for this step.

In [ ]:
! conda config --add channels conda-forge
! conda install -y mamba python==3.9
! mamba update -qy --all
! mamba clean -qafy

Solving environment: \ failed with initial frozen solve. Retrying with flexible solve.
Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - 

4. To install and activate genomenlp, run the following commands:

If you are performing this in google colab, please allow 10 minutes for this step.

In [ ]:
! mamba install -c tyronechen -c conda-forge genomenlp==2.8.4 ipykernel -y
! pip install -U datasets  # tested with 2.15 (thank you anonymous K-CAP 2023 audience member)
# after the above completes
! sweep -h
# you should see some output


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.2.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['genomenlp==2.8.4', 'ipykernel']

[+] 0.0s
[+] 0.1s
tyronechen/linux-64   ⣾  
tyronechen/noarch    

## 4. Setting up a biological dataset
Understanding of the data and experimental design is a necessary first step to analysis. In our case study, we perform a simple two case classification, where the dataset consists of a corpora of biological sequence data belonging to two categories. Genomic sequence associated with promoters and non-promoter regions are available. In the context of biology, promoters are important modulators of gene expression, and most are relatively short as well as information rich. Motif prediction is an active, on-going area of research in biology, since many of these signals are weak and difficult to detect, as well as varying in frequency and distribution across different species. **Therefore, our aim is to classify sequences into promoter and non-promoter sequence categories.**

*NOTE: [A more detailed description of the data is available here](https://github.com/khanhlee/bert-promoter).*

Our data is available in the form of `fasta` files. `fasta` files are a common format for storing biological sequence data. They typically contain headers that provide information about the sequence, followed by the sequence itself. They can also store other nucleic acid data, as well as protein. The fasta format contains headers with a leading `>`. Lines without `>` contain biological sequence data and can be newline separated. In our simple example, the full set of characters are the DNA nucleotides adenine `A`, thymine `T`, cytosine `C` and guanine `G`. These are the building blocks of the genetic code.

The files can be downloaded here for [non promoter sequences](https://raw.githubusercontent.com/khanhlee/bert-promoter/main/data/non_promoter.fasta) and [promoter sequences](https://raw.githubusercontent.com/khanhlee/bert-promoter/main/data/promoter.fasta).

```
# HEADER:   >PCK12019 FORWARD 639002 STRONG
  SEQUENCE: TAGATGTCCTTGATTAACACCAAAAT
  HEADER:   >ECK12066 REVERSE 3204175 STRONG
  SEQUENCE: AAAGAAAATAATTAATTTTACAGCTG
```

`NOTE: In real world data, other characters are available which refer to multiple possible nucleotides, for example ``W`` indicates either an ``A`` or a ``T``. RNA includes the character ``U``, and proteins include additional letters of the alphabet.`

Tokenisation in genomics involves segmenting biological sequences into smaller units, called tokens (or k-mers in biology) for further processing. In the context of genomics, tokens can represent individual nucleotides, k-mers, codons, or other biologically meaningful segments. Just as in conventional NLP, tokenisation is required to facilitate most downstream operations.

Here, we provide gzipped fasta file(s) as input. While conventional biological tokenisation splits a sequence into arbitrary-length segments, empirical tokenisation derives the resulting tokens directly from the corpus, with vocabulary size as the only user-defined parameter. Data is then split into training, testing and/or validation partitions as desired by the user and automatically reformatted for input into the deep learning pipeline.

`NOTE: We provide the conventional k-merisation method as well as an option for users. In our pipeline specifically, the empirical tokenisation and data object creation is split into two steps, while k-merisation combines both in one operation. This is due to the empirical tokenisation process having to “learn” tokens from the data.`

To download the 'fasta' files directly from their URL, run the following commands:

In [ ]:
! wget 'https://raw.githubusercontent.com/khanhlee/bert-promoter/main/data/promoter.fasta'
! wget 'https://raw.githubusercontent.com/khanhlee/bert-promoter/main/data/non_promoter.fasta'

--2023-10-04 03:15:02--  https://raw.githubusercontent.com/khanhlee/bert-promoter/main/data/promoter.fasta
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 448725 (438K) [text/plain]
Saving to: ‘promoter.fasta’

promoter.fasta      100%[===================>] 438.21K  --.-KB/s    in 0.05s   

2023-10-04 03:15:02 (8.74 MB/s) - ‘promoter.fasta’ saved [448725/448725]

--2023-10-04 03:15:02--  https://raw.githubusercontent.com/khanhlee/bert-promoter/main/data/non_promoter.fasta
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 303273 (296

In [ ]:
# Empirical tokenisation pathway
! tokenise_bio -i promoter.fasta non_promoter.fasta -t tokens.json
# -i INFILE_PATHS path to files with biological seqs split by line
# -t TOKENISER_PATH path to tokeniser.json file to save or load data

COMMAND LINE ARGUMENTS FOR REPRODUCIBILITY:

	 /usr/local/bin/tokenise_bio -i promoter.fasta non_promoter.fasta -t tokens.json 

[00:00:00] Pre-processing sequences                 ████████ 0        /        0
[00:00:00] Pre-processing sequences                 ████████ 5000     /        0
[00:00:00] Pre-processing sequences                 ████████ 0        /        0
[00:00:00] Suffix array seeds                       ████████ 0        /        0
[00:00:00] Suffix array seeds                       ████████ 6764     /     6764

[00:00:00] EM training                              ████████ 6764     /       16
[00:00:00] EM training                              ████████ 6764     /       16
[00:00:07] EM training                              ░░░░░░░░ 1        /       16
[00:00:10] EM training                              █░░░░░░░ 2        /       16
[00:00:10] EM training                              ████████ 16       /       16



This generates a `json` file with tokens and their respective weights or IDs
```
# [00:00:00] Pre-processing sequences
[00:00:00] Suffix array seeds
[00:00:14] EM training
Sample input sequence: AACCGGTT
Sample tokenised: [156, 2304]
Token: : k-mer map: 156  : : AA
Token: : k-mer map: 2304 : : CCGGTT
```

## 5. Format a dataset for input into genomeNLP
In this section, we reformat the data to meet the requirements of our pipeline which takes specifically structured inputs. This intermediate data structure serves as the foundation for downstream analyses and facilitates seamless integration with the pipeline. Our pipeline contains a method that performs this automatically, generating a reformatted dataset with the desired structure.

`NOTE: The data format is identical to that used by the HuggingFace ``datasets`` and ``transformers`` libraries.`







In [ ]:
# Empirical tokenisation pathway
! create_dataset_bio promoter.fasta non_promoter.fasta tokens.json  -o data.csv
# -o OUTFILE_DIR write dataset to directory as
#   [ csv \| json \| parquet \| dir/ ] (DEFAULT:"hf_out/")
# default datasets split: train 90%, test 5% and validation set 5%

COMMAND LINE ARGUMENTS FOR REPRODUCIBILITY:

	 /usr/local/bin/create_dataset_bio promoter.fasta non_promoter.fasta tokens.json -o data.csv 

/usr/local/lib/python3.9/site-packages/genomenlp/utils.py:840: UserWarning: Any commas in fasta headers will be replaced with __!
  warn("Any commas in fasta headers will be replaced with __!")
Extracting data files: 100% 1/1 [00:00<00:00, 1637.12it/s]
Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-2e6dcbb283e7eef8/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.

USING EXISTING TOKENISER: tokens.json

ASSIGNING CLASS LABELS:
                                                             
DATASET FEATURES:
 {'idx': Value(dtype='string', id=None), 'feature': Value(dtype='string', id=None), 'labels': ClassLabel(names=['NEGATIVE', 'POSITIVE'], id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'token_type_ids': Se

The output is a reformatted dataset containing the same information. Properties required for a typical machine learning pipeline are added, including labels, customisable data splits and token identifiers.

```
# DATASET AFTER SPLIT:
DatasetDict ({
  train: Dataset ({
  features: ['idx', 'feature', 'labels', 'input_ids', 'token_type_ids', 'attention_mask’],
  num_rows: 12175 })
  test: Dataset ({
  features: ['idx', 'feature', 'labels', 'input_ids', 'token_type_ids', 'attention_mask’],
  num_rows: 677 })
  valid: Dataset ({
  features: ['idx', 'feature', 'labels', 'input_ids', 'token_type_ids', 'attention_mask’],
  num_rows: 676 })
})
```
`NOTE: The column token_type_ids is not actually needed in this specific case study, but it is safely ignored in such cases.`

```
# SAMPLE TOKEN MAPPING FOR FIRST 5 TOKENS IN SEQ:
TOKEN ID: 858  | TOKEN: TCA
TOKEN ID: 2579 | TOKEN: GCATCAC
TOKEN ID: 111  | TOKEN: TATT
TOKEN ID: 99   | TOKEN: CAGG
TOKEN ID: 777  | TOKEN: AGGCT
```

## 6. Preparing a hyperparameter sweep
In machine learning, achieving optimal model performance often requires finding the right combination of hyperparameters (assuming the input data is viable). Hyperparameters vary depending on the specific algorithm and framework being used, but commonly include learning rate, dropout rate, batch size, number of layers and optimiser choice. These parameters heavily influence the learning process and subsequent performance of the model.

For this reason, hyperparameter sweeps are normally carried out to systematically test combinations of hyperparameters, with the end goal of identifying the configuration that produces the best model performance. Usually, sweeps are carried out on a small partition of the data only to maximise efficiency of compute resources, but it is not uncommon to perform sweeps on entire datasets. Various strategies, such as grid search, random search, or bayesian optimisation, can be employed during a hyperparameter sweep to sample parameter values. Additional strategies such as early stopping can also be used.

To streamline the hyperparameter optimization process, we use the `wandb` (Weights & Biases) platform which has a user-friendly interface and powerful tools for tracking experiments and visualising results.

First, sign up for a wandb account at: https://wandb.ai/site and login by pasting your API key.



In [ ]:
! wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Now, we use the `sweep` tool to perform hyperparameter sweep. Search strategy, parameters and search space are passed in as a `json` file.

```
# sweep parameters
{
  "method": "random",
  "name": "sweep",
  "metric": {
    "goal": "maximize",
    "name": "eval/f1"
  },
  "parameters": {
    "batch_size": {"values": [5, 10, 15]},
    "epochs": {"values": [1, 2, 3, 4, 5]},
    "learning_rate": {"max": 0.1, "min": 0.0001}
  }
}
```

In [ ]:
! wget 'https://raw.githubusercontent.com/Navya003/Navya003/main/params.json'

In [ ]:
! sweep data_out/train.parquet parquet  tokens.json -t data_out/test.parquet  -v  data_out/valid.parquet -w params.json -e tyagilab -p dna.colab -g dna.sweep.1  -l labels -n 8 --partition_percent 10
# -t TEST, path to [ csv \| csv.gz \| json \| parquet ] file
# -v VALID, path to [ csv \| csv.gz \| json \| parquet ] file
# -w HYPERPARAMETER_SWEEP, run a hyperparameter sweep with config from file
# -e ENTITY_NAME, wandb team name (if available).
# -p PROJECT_NAME, wandb project name (if available)
# -l LABEL_NAMES, provide column with label names (DEFAULT: "").
# -n SWEEP_COUNT, run n hyperparameter sweeps

wandb: Currently logged in as: navya-tyagi (tyagilab). Use `wandb login --relogin` to force relogin
/usr/local/lib/python3.9/site-packages/genomenlp/sweep.py:132: UserWarning: Training on cpu but fp16 is on. Disabled as incompatible.
  warn("Training on cpu but fp16 is on. Disabled as incompatible.")


USING DEVICE:
 cpu


ARGUMENTS:
 Namespace(train='data.csv/train.parquet', format='parquet', tokeniser_path='tokens.json', test='data.csv/test.parquet', valid='data.csv/valid.parquet', model='distilbert', model_features=None, output_dir='./sweep_out', device='auto', vocab_size=32000, hyperparameter_sweep='params.json', label_names=['labels'], sweep_count=8, entity_name='tyagilab', project_name='dna.colab', group_name='dna.sweep.1', partition_percent=10, metric_opt='eval/f1', resume_sweep=None, fp16_off=True, wandb_off=True, report_to='wandb') 


USING EXISTING TOKENISER: tokens.json
Extracting data files: 100% 3/3 [00:00<00:00, 1604.55it/s]
Dataset parquet downloaded and prepared to /roo

The output is written to the specified directory, in this case `sweep_out` and will contain the output of a standard `pytorch` saved model, including some `wandb` specific output.

```
 *****Running training*****
Num examples = 12175
Num epochs= 1
Instantaneous batch size per device = 64
Total train batch size per device = 64
Gradient Accumulation steps= 1
Total optimization steps= 191
```
The sweeps gets synced to the wandb dashboard along with various interactive custom charts and tables which we provide as part of our pipeline. A small subset of plots are provided for reference. Interactive versions of these and more plots are available on wandb.

![image.png](https://genomenlp.readthedocs.io/en/latest/_images/sweep_conf_mat.png)
![image.png](https://genomenlp.readthedocs.io/en/latest/_images/sweep_pr.png)
![image.png](https://genomenlp.readthedocs.io/en/latest/_images/sweep_roc.png)
![image.png](https://genomenlp.readthedocs.io/en/latest/_images/sweep_f1.png)
![image.png](https://genomenlp.readthedocs.io/en/latest/_images/sweep_loss.png)
![image.png](https://genomenlp.readthedocs.io/en/latest/_images/sweep_lr.png)

[Here is an example of a full wandb generated report:](https://api.wandb.ai/links/tyagilab/a56uxmff)

You may inspect your own generated reports after they complete.

## 7. Selecting optimal hyperparameters for training
Having completed a sweep, we next identified the best set of parameters for model training. We do this by examining training metrics. These serve as quantitative measures of a model’s performance during training. These metrics provide insights into the model’s accuracy and generalisation capabilities. We explore commonly used training metrics, including accuracy, loss, precision, recall, and `f1` score to inform us of a model’s performance

A key event we want to avoid is overfitting. Overfitting occurs when a learning model performs exceptionally well on the training data but fails to generalise to unseen data, making it unfit for use outside of the specific scope of the experiment. This can be detected by observing performance metrics, if the accuracy decreases and later increases an overfit event has occurred. In real world applications, this can lead to adverse events that directly impact us, considering that such models are used in applications such as drug prediction or self-driving cars. Here, we use the `f1` score calculated on the testing set as the main metric of interest. We showed that we obtain a best `f1` score of `0.79`.
```
Best run revived-sweep-6 with eval/f1=0.7900291349379833
BEST MODEL AND CONFIG FILES SAVED TO: *./sweep_out/model_files*
HYPERPARAMETER SWEEP END
```

[Here is an example of a full wandb generated report for the “best” run](https://api.wandb.ai/links/tyagilab/opr8yd88).

You may inspect your own generated reports after they complete.

## 8. With the selected hyperparameters, train the full dataset
In a conventional workflow, the sweep is performed on a small subset of training data. The resulting parameters are then recorded and used in the actual training step on the full dataset. Here, we perform the sweep on the entire dataset, and hence remove the need for further training. If you perform this on your own data and want to use a small subset, you can do so and then pass the recorded hyperparameters with the same input data to the train function of the pipeline. We include an example of this below for completeness, but you can skip this for our specific case study. Note that the input is almost identical to `sweep`.

In [ ]:
! train data.csv/train.parquet parquet  tokens.json -t data.csv/test.parquet  -v data.csv/valid.parquet --hyperparameter_file params.json -e tyagilab -p dna.colab -g dna.train --label_names labels --config_from_run tyagilab/dna.colab/kk2g9pxv --output_dir train.out
# -t TEST, path to [ csv \| csv.gz \| json \| parquet ] file
# -v VALID, path to [ csv \| csv.gz \| json \| parquet ] file
# -w HYPERPARAMETER_SWEEP, run a hyperparameter sweep with config from file
# -e ENTITY_NAME, wandb team name (if available).
# -p PROJECT_NAME, wandb project name (if available)
# -l LABEL_NAMES, provide column with label names (DEFAULT: "").
# -n SWEEP_COUNT, run n hyperparameter sweeps

wandb: Currently logged in as: navya-tyagi (tyagilab). Use `wandb login --relogin` to force relogin


USING DEVICE:
 cpu


ARGUMENTS:
 Namespace(output_dir='train.out', overwrite_output_dir=False, do_train=False, do_eval=False, do_predict=False, evaluation_strategy='no', prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, eval_delay=0, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, lr_scheduler_type='linear', warmup_ratio=0.0, warmup_steps=0, log_level='passive', log_level_replica='passive', log_on_each_node=True, logging_dir=None, logging_strategy='steps', logging_first_step=False, logging_steps=500, logging_nan_inf_filter=True, save_strategy='steps', save_steps=500, save_total_limit=None, save_on_each_node=False, no_cuda=False, 

The output is written to the specified directory, in this case `train_out` and will contain the output of a standard `pytorch` saved model, including some `wandb` specific output.

The trained model gets synced to the `wandb` dashboard along with various interactive custom charts and tables which we provide as part of our pipeline. A small subset of plots are provided for reference. Interactive versions of these and more plots are available on `wandb`.

![image.png](https://genomenlp.readthedocs.io/en/latest/_images/train_conf_mat.png)
![image.png](https://genomenlp.readthedocs.io/en/latest/_images/train_pr.png)
![image.png](https://genomenlp.readthedocs.io/en/latest/_images/train_roc.png)
![image.png](https://genomenlp.readthedocs.io/en/latest/_images/train_f1.png)
![image.png](https://genomenlp.readthedocs.io/en/latest/_images/train_loss.png)
![image.png](https://genomenlp.readthedocs.io/en/latest/_images/train_lr.png)

[Here is an example of a full wandb generated report:](https://api.wandb.ai/links/tyagilab/opr8yd88)

You may inspect your own generated reports after they complete.

## 9. Perform cross-validation
Having identified the best set of parameters and trained the model, we next want to conduct a comprehensive review of data stability, and we do this by evaluating model performance across different data slices. This assessment is known as cross-validation. We make use of k-fold cross-validation in which data is divided into k subsets and the model is trained and tested on these individual subsets.

In [ ]:
! cross_validate data.csv/train.parquet parquet  --tokeniser_path tokens.json -t data.csv/test.parquet  -v  data.csv/valid.parquet -f params.json -e tyagilab -p dna.colab -g dna.cval -l labels -m sweep_out --config_from_run tyagilab/dna.colab/kk2g9pxv --output_dir cval.out
# --config_from_run WANDB_RUN_ID, *best run id*
# –-output_dir OUTPUT_DIR
# -l label_names
# -k KFOLDS, run n number of kfolds

Streaming output truncated to the last 5000 lines.
  2% 10/508 [00:02<02:40,  3.09it/s]
  2% 11/508 [00:03<02:41,  3.08it/s]
  2% 12/508 [00:03<02:41,  3.07it/s]
  3% 13/508 [00:03<02:40,  3.08it/s]
  3% 14/508 [00:04<02:40,  3.08it/s]
  3% 15/508 [00:04<02:41,  3.06it/s]
  3% 16/508 [00:04<02:41,  3.05it/s]
  3% 17/508 [00:05<02:41,  3.04it/s]
  4% 18/508 [00:05<02:40,  3.05it/s]
  4% 19/508 [00:05<02:44,  2.98it/s]
  4% 20/508 [00:06<02:40,  3.03it/s]
  4% 21/508 [00:06<02:24,  3.37it/s]
  4% 22/508 [00:06<02:12,  3.66it/s]
  5% 23/508 [00:06<02:02,  3.95it/s]
  5% 24/508 [00:07<01:57,  4.13it/s]
  5% 25/508 [00:07<01:51,  4.31it/s]
  5% 26/508 [00:07<01:50,  4.38it/s]
  5% 27/508 [00:07<01:48,  4.42it/s]
  6% 28/508 [00:07<01:44,  4.58it/s]
  6% 29/508 [00:08<01:43,  4.61it/s]
  6% 30/508 [00:08<01:41,  4.70it/s]
  6% 31/508 [00:08<01:42,  4.66it/s]
  6% 32/508 [00:08<01:41,  4.68it/s]
  6% 33/508 [00:09<01:41,  4.70it/s]
  7% 34/508 [00:09<01:41,  4.68it/s]
  7% 35/508 [00:09<01:41

```
*****Running training*****
Num examples = 10653
Num epochs= 2
Instantaneous batch size per device = 16
Total train batch size (w, parallel, distributed & accumulation)= 16
Gradient Accumulation steps= 1
Total optimization steps= 1332
Automatic Weights & Biases logging enabled
```
The cross-validation runs are uploaded to the wandb dashboard along with various interactive custom charts and tables which we provide as part of our pipeline. These are conceptually identical to those generated by sweep or train. A small subset of plots are provided for reference. Interactive versions of these and more plots are available on wandb.

![image.png](https://genomenlp.readthedocs.io/en/latest/_images/cval_conf_mat.png)
![image.png](https://genomenlp.readthedocs.io/en/latest/_images/cval_pr.png)
![image.png](https://genomenlp.readthedocs.io/en/latest/_images/cval_roc.png)
![image.png](https://genomenlp.readthedocs.io/en/latest/_images/cval_f1.png)
![image.png](https://genomenlp.readthedocs.io/en/latest/_images/cval_loss.png)
![image.png](https://genomenlp.readthedocs.io/en/latest/_images/cval_lr.png)

[Here is an example of a full wandb generated report:](https://api.wandb.ai/links/tyagilab/8vony79x)

You may inspect your own generated reports after they complete.

## 10. Compare different models
The aim of this step is to compare performance of different deep learning models efficiently while avoiding computationally expensive re-training and data download in conventional model comparison. In the case of patient data, they are often inaccessible for privacy reasons, and in other cases they are not uploaded by the authors of the experiment.

For the purposes of this simple case study, we compare multiple sweeps of the same dataset as a demonstration. In a real life application, existing biological models can be compared against the user-generated one.

In [ ]:
! fit_powerlaw tyagilab/dna.colab/kk2g9pxv -o fit
# -o OUTPUT_DIR, path to output metrics directory

COMMAND LINE ARGUMENTS FOR REPRODUCIBILITY:

	 /usr/local/bin/fit_powerlaw tyagilab/dna.colab/kk2g9pxv -o fit 

Loading model from: fit/kk2g9pxv
Some weights of the model checkpoint at fit/kk2g9pxv were not used when initializing DistilBertModel: ['classifier.weight', 'classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


python      version 3.9.0 | packaged by conda-forge | (default, Nov 26 2020, 07:57:39) 
[GCC 9.3.0]
numpy       version 1.26.0
torch version 1.10.0
INFO:weightwatcher:

python      version 3.9.0 | 

This tool outputs a variety of plots in the specified directory.
```
$ ls fit
> alpha_hist.pdf  alpha_plot.pdf  kk2g9pxv
```   
Very broadly, the overlaid bar plots allow the user to compare the performance of different models on the same scale. A narrow band around 2-5 with few outliers is in general cases an indicator of good model performance. This is a general guideline and will differ depending on context! [For a detailed explanation of these plots, please refer to the original publication.](https://arxiv.org/pdf/2202.02842.pdf)

![image.png](https://genomenlp.readthedocs.io/en/latest/_images/alpha_hist.png)
![image.png](https://genomenlp.readthedocs.io/en/latest/_images/alpha_plot.png)


In [ ]:
! ls fit

alpha_hist.pdf	alpha_plot.pdf	kk2g9pxv


## 11. Obtain model interpretability scores
Model interpretability is often used for debugging purposes, by allowing the user to “see” (to an extent) what a model is focusing on. In this case, the tokens which contribute to a certain classification are highlighted. The green colour indicates a classification towards the target category, while the red colour indicates a classification away from the target category. Colour intensity indicates the classification score.

In some scenarios, we can exploit this property by identifying regulatory regions or motifs in DNA sequences, or discovering amino acid residues in protein structure critical to its function, leading to a deeper understanding of the underlying biological system.

In [ ]:
! gzip promoter.fasta
! gzip -cd promoter.fasta.gz | head -n10 > subset.fasta
! interpret tyagilab/dna.colab/kk2g9pxv subset.fasta -o model_interpret



ARGUMENTS:
 Namespace(model_path='tyagilab/dna.colab/kk2g9pxv', input_seqs=['subset.fasta'], tokeniser_path='', output_dir='model_interpret', label_names=None, pdf_options=None) 


Use tokeniser from: model_interpret
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Calculating class attributions:   0% 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.9/site-packages/genomenlp/utils.py:1528: UserWarning: Both pdfkit and wkhtmltopdf are required. Please refer to https://github.com/JazzCore/python-pdfkit for install instructions. No pdf output will be generated!
  warn(warning_message)
Calculating class attributions: 100% 1/1 [00:12<00:00, 12.65s/it]


```
ECK120010480 CSGDP1 REVERSE 1103344 SIGMA38.html
ECK120010489 OSMCP2 FORWARD 1556606 SIGMA38.html
ECK120010491 TOPAP1 FORWARD 1330980 SIGMA32 STRONG.html
ECK120010496 YJAZP  FORWARD 4189753 SIGMA32 STRONG.html
ECK120010498 YADVP2 REVERSE 156224  SIGMA38.html
```
![image.png](https://genomenlp.readthedocs.io/en/latest/_images/ECK120009966.png)
![image.png](https://genomenlp.readthedocs.io/en/latest/_images/ECK120016719.png)

## Citation
Cite our manuscript here:
```
@article{chen2023genomicbert,
    title={genomicBERT and data-free deep-learning model evaluation},
    author={Chen, Tyrone and Tyagi, Navya and Chauhan, Sarthak and Peleg, Anton Y and Tyagi, Sonika},
    journal={bioRxiv},
    month={jun},
    pages={2023--05},
    year={2023},
    publisher={Cold Spring Harbor Laboratory},
    doi={10.1101/2023.05.31.542682},
    url={https://doi.org/10.1101/2023.05.31.542682}
}
```

Cite our software here:
```
@software{tyrone_chen_2023_8135591,
  author       = {Tyrone Chen and
                  Navya Tyagi and
                  Sarthak Chauhan and
                  Anton Y. Peleg and
                  Sonika Tyagi},
  title        = {{genomicBERT and data-free deep-learning model
                  evaluation}},
  month        = jul,
  year         = 2023,
  publisher    = {Zenodo},
  version      = {latest},
  doi          = {10.5281/zenodo.8135590},
  url          = {https://doi.org/10.5281/zenodo.8135590}
}
```